## threading.Event

참고 : https://docs.python.org/3/library/threading.html#event-objects

thread에게 signal을 보내는 flag

.set : flag => True // .clear : flag => False

.wait() : flag가 True가 되기까지 대기

In [5]:
import threading
import time

def my_func(event):
    print("Thread Start")
    # event가 set될 때 까지 무한 수행
    while not event.is_set():
        pass
    print("Thread End")

event = threading.Event()
t1 = threading.Thread(target=my_func,args=(event,))
t1.start()

time.sleep(3) # 3초 뒤에 thread가 종료되도록 signal 설정
event.set() # Event => True

Thread Start
Thread End


## threading.Semaphore - counter

참고 : http://blog.naver.com/PostView.nhn?blogId=wwwkasa&logNo=220952228840&categoryNo=58&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView

동시에 실행가능한 thread 수를 제한하는 역할을 한다. 

사용 예 : 많은 thread가 실행중일 때(e.g. connections) 서버의 Resource 관리를 위해 특정 작업에 대해서는 실행 가능한 thread 수를 제한한다.

there is a guarantee that the operating system will not swap out the thread in the middle of incrementing or decrementing the counter.

.acquire : counter 감소

.relase : counter 증가

acquire를 실행하지 못하면 대기 상태가 된다.

In [18]:
import threading
import logging
import time

class Semaphor(threading.Thread):
    sema = threading.Semaphore(2) # counter 값 지정
    
    def __init__(self, name, daemon):
        threading.Thread.__init__(self, name=name, daemon=daemon)
    def run(self):
        Semaphor.sema.acquire() # counter -= 1
        print(f"{self.name} working")
        time.sleep(2)
        Semaphor.sema.release() # counter += 1
        print(f"{self.name} releasing")
        
thread_list = []
for i in range(1,6):
    thread_list.append(Semaphor(i,True))
for t in thread_list:
    t.start()
time.sleep(2) # 2개씩만 돌아가는 모습 확인 가능

1 working
2 working
1 releasing3 working

2 releasing
4 working
3 releasing5 working

4 releasing
5 releasing


## threading.barrier

참고 : https://www.geeksforgeeks.org/barrier-objects-python/

Threads all will remain blocked until the specified number of threads are waiting, and then the are all released at the same time.

사용 예 : Multi-thread 사용 시 initializing 작업을 모두 끝내고 동시에 시작할 수 있도록 한다.

(동시 시작은 단순 OS에게 thread의 본격적인 작업을 시작하라는 알리는 것 뿐 GIL에 따라 수행된다.)

parties : 기다리게 할 thread 수    
n_waiting : 현재 대기중인 thread 수

.wait()을 실행할 때마다 counter가 증가하고 parties의 수와 일치하면 모든 thread 시작

In [5]:
import threading
import logging
import time

class Barrier(threading.Thread):
    barrier = threading.Barrier(3) # 기다릴 parties 지정
    
    def __init__(self, name):
        threading.Thread.__init__(self, name=name)
    def run(self):
        print(self.name, "initializing...")
        print(self.name, "waiting")
        Barrier.barrier.wait() # barrier에 추가
        print(self.name, "start")

logging.getLogger().setLevel(logging.DEBUG)

thread1 = Barrier("Thread 1")
thread2 = Barrier("Thread 2")
thread1.start()
thread2.start()

time.sleep(2)
logging.debug(f"num of parties : {Barrier.barrier.parties}")
logging.debug(f"waiting Thread Number : {Barrier.barrier.n_waiting}")
logging.info("barrier start now")
Barrier.barrier.wait() # waiting 수와 parties가 같아짐
thread1.join()

Thread 1Thread 2 initializing...
Thread 2 waiting
 initializing...
Thread 1 waiting


DEBUG:root:num of parties : 3
DEBUG:root:waiting Thread Number : 2
INFO:root:barrier start now


Thread 2 start
Thread 1 start


## threading.Timer

특정 시간 후에 작동하도록 설정 가능

.cancel : 중지

In [3]:
import threading
import time

def my_func():
    print("working")
    
start = time.time()
print(start)
t = threading.Timer(5, my_func)
t.start()
t.join()
print(time.time()-start)

1555594829.9424112
working
5.001625061035156
